# Main Notebook

## Summary

This notebook will do following steps:
 -  Pull validation data
 -  Load CTR predictions from pickles for X different models
 -  Load BaseBid Predictions from pickels for X different models
 -  For X models combine the above two steps to build X BidPrices for each impression
 -  Hold auction for each impression one by one
 -  Summary Scores for each of X models 

# Preparation

In [74]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from random import randint    
import seaborn as sns
from score_models import Scorer

%matplotlib inline

# Pull Validation Data

In [75]:
path = "C:/Users/Akis-/OneDrive/Masters/Web Economics/Data/"
#test_df = pd.read_csv(path+'test.csv')
#train_df = pd.read_csv(path+'train.csv')
validation_df = pd.read_csv(path+'validation.csv')

In [76]:

#print(validation_df.head())
print(validation_df.shape[0])
print(validation_df.loc[validation_df["click"] == 1].shape[0])
avgCTR =validation_df.loc[validation_df["click"] == 1].shape[0]/(validation_df.shape[0])
print(avgCTR)

303925
202
0.0006646376573167722


# Load CTR Predictions

Need a function that will combine multiple dataframes into one

In [77]:
teamAdf = pd.DataFrame(columns=[],index=range(0,10,1))
teamAdf["teamA_CTR"]=np.random.randint(0, 10, teamAdf.shape[0])/10
teamBdf = pd.DataFrame(columns=[],index=range(0,10,1))
teamBdf["CTR"]=np.random.randint(0, 10, teamAdf.shape[0])/10
print(teamAdf)
print(teamBdf)




resultCTR=teamAdf
resultCTR["teamB_CTR"]=teamBdf["CTR"]
#result = pd.merge(teamAdf, teamBdf, how='inner', on=None, left_on=None, right_on=None, left_index=False, right_index=False, sort=False, suffixes=('_x', '_y'), copy=True, indicator=False, validate=None)
print(resultCTR)

   teamA_CTR
0        0.2
1        0.9
2        0.3
3        0.1
4        0.5
5        0.4
6        0.4
7        0.4
8        0.6
9        0.7
   CTR
0  0.9
1  0.5
2  0.1
3  0.4
4  0.2
5  0.1
6  0.3
7  0.7
8  0.1
9  0.8
   teamA_CTR  teamB_CTR
0        0.2        0.9
1        0.9        0.5
2        0.3        0.1
3        0.1        0.4
4        0.5        0.2
5        0.4        0.1
6        0.4        0.3
7        0.4        0.7
8        0.6        0.1
9        0.7        0.8


# Load Basebid Predictions

Need a function that will combine multiple dataframes into one

In [78]:
teamABaseBid = pd.DataFrame(columns=[],index=range(0,10,1))
teamABaseBid["teamA_BaseBid"]=np.random.randint(227, 301, teamABaseBid.shape[0])
teamBBaseBid = pd.DataFrame(columns=[],index=range(0,10,1))
teamBBaseBid["teamB_BaseBid"]=np.random.randint(227, 301, teamBBaseBid.shape[0])
print(teamABaseBid)
print(teamBBaseBid)


resultBaseBid=teamABaseBid
resultBaseBid["teamB_BaseBid"]=teamBBaseBid["teamB_BaseBid"]
#result = pd.merge(teamAdf, teamBdf, how='inner', on=None, left_on=None, right_on=None, left_index=False, right_index=False, sort=False, suffixes=('_x', '_y'), copy=True, indicator=False, validate=None)
print(resultBaseBid)

   teamA_BaseBid
0            268
1            300
2            288
3            264
4            250
5            229
6            266
7            287
8            257
9            292
   teamB_BaseBid
0            268
1            229
2            253
3            263
4            245
5            300
6            227
7            290
8            289
9            258
   teamA_BaseBid  teamB_BaseBid
0            268            268
1            300            229
2            288            253
3            264            263
4            250            245
5            229            300
6            266            227
7            287            290
8            257            289
9            292            258


# Build Bid Prices for all models

Need a function that will build a new dataframe conatining bids of all teams by combining the previous two dataframes.

In [79]:
resultBids = resultBaseBid.values * ((resultCTR*(138/35000))/avgCTR)
resultBids2 = resultBaseBid.values * (resultCTR/0.5)
#print(resultBids)
print(resultBids2)

   teamA_CTR  teamB_CTR
0      107.2      482.4
1      540.0      229.0
2      172.8       50.6
3       52.8      210.4
4      250.0       98.0
5      183.2       60.0
6      212.8      136.2
7      229.6      406.0
8      308.4       57.8
9      408.8      412.8


# Hold Auction for each impression one-by-one

Need a function that will step through the impressions one by one checking who won the impression, whilst keeping track of each player's budget.

In [80]:

resultBids2['click']=np.random.randint(0, 1, resultBids2.shape[0])
resultBids2[['teamA_CTR','teamB_CTR']]
#Scorer()

,teamA_CTR,teamB_CTR
0,107.2,482.4
1,540.0,229.0
2,172.8,50.6
3,52.8,210.4
4,250.0,98.0
5,183.2,60.0
6,212.8,136.2
7,229.6,406.0
8,308.4,57.8
9,408.8,412.8


In [81]:
s = Scorer()
s.set_df(resultBids2, ['teamA_CTR', 'teamB_CTR'],1500,200)

   teamA_CTR  teamB_CTR  click  winner  price
0      107.2      482.4      0       1  200.0
1      540.0      229.0      0       0  229.0
2      172.8       50.6      0       0  200.0
3       52.8      210.4      0       1  200.0
4      250.0       98.0      0       0  200.0


# Scores

Summarise the main metrics for each team.